In [57]:
import pandas as pd
import numpy as np

In [58]:
movie = pd.read_csv(r"C:\Users\yasha\Desktop\movie_recommender\Backend\ml\data\processed\movies_clean.csv")
movies = movie

In [59]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260932 entries, 0 to 1260931
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1260932 non-null  int64  
 1   title                 1260932 non-null  object 
 2   vote_average          1260932 non-null  float64
 3   vote_count            1260932 non-null  int64  
 4   status                1260932 non-null  object 
 5   release_date          1025808 non-null  object 
 6   adult                 1260932 non-null  bool   
 7   imdb_id               629446 non-null   object 
 8   popularity            1260932 non-null  float64
 9   genres                718584 non-null   object 
 10  production_countries  666319 non-null   object 
 11  combined_text         1260932 non-null  object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 107.0+ MB


In [60]:
# Convert all object/string columns to lowercase
for col in movies.select_dtypes(include='object').columns:
    movies[col] = movies[col].str.lower()


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
vector = TfidfVectorizer(stop_words='english', max_features=20000)
model = vector.fit_transform(movie['combined_text'])

In [63]:
print(model)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 26052753 stored elements and shape (1260932, 20000)>
  Coords	Values
  (0, 8890)	0.5594744822978015
  (0, 490)	0.0586533791445
  (0, 15669)	0.07205555520500254
  (0, 6755)	0.07204009157601167
  (0, 599)	0.06840357024087292
  (0, 3613)	0.14630503810872506
  (0, 16371)	0.12528270017741225
  (0, 17912)	0.10352711782753088
  (0, 3775)	0.12003893745693847
  (0, 4162)	0.11436934144358318
  (0, 6183)	0.12361461949578016
  (0, 17236)	0.3902046495055326
  (0, 17676)	0.1262728491638708
  (0, 12512)	0.11263941554650293
  (0, 3178)	0.0857653086169367
  (0, 14614)	0.11918482637907717
  (0, 12544)	0.061098902649629534
  (0, 10356)	0.04943570873619671
  (0, 13085)	0.13494662017716338
  (0, 17683)	0.10636816669051126
  (0, 3987)	0.10475729077014945
  (0, 8851)	0.10336290728937528
  (0, 13227)	0.08501012592905628
  (0, 8716)	0.09376879443270304
  (0, 17673)	0.10854809697236831
  :	:
  (1260930, 530)	0.16254300640089395
  (1260930, 3982)	0.17

In [64]:
from sklearn.decomposition import TruncatedSVD

In [65]:
svd = TruncatedSVD(n_components=200, random_state=42)
vector_reduced = svd.fit_transform(model)

In [66]:
vector_reduced

array([[ 0.11169659, -0.00835709, -0.02147402, ..., -0.00937292,
         0.00469239,  0.00106571],
       [ 0.13026337, -0.02335916, -0.03939034, ..., -0.0192442 ,
         0.00219661,  0.00062995],
       [ 0.1146324 , -0.02247688, -0.00763688, ..., -0.04158831,
         0.03174735, -0.03136228],
       ...,
       [ 0.0773334 , -0.00770446, -0.00986875, ...,  0.03765192,
         0.03533078, -0.02812561],
       [ 0.097686  , -0.01263968, -0.0310992 , ...,  0.0157602 ,
         0.01949153, -0.02555751],
       [ 0.02536952, -0.00067942, -0.00990235, ..., -0.01606142,
         0.0112789 , -0.01568437]], shape=(1260932, 200))

In [67]:
# import pickle

In [68]:
# pickle.dump(vector, open(r"C:\Users\yasha\Desktop\movie_recommender\Backend\ml\models\vector.pkl", "wb"))
# pickle.dump(model, open(r"C:\Users\yasha\Desktop\movie_recommender\Backend\ml\models\model.pkl", "wb"))
# pickle.dump(vector_reduced, open(r"C:\Users\yasha\Desktop\movie_recommender\Backend\ml\models\vector_reduced.pkl", "wb"))


In [69]:
from sklearn.metrics.pairwise import cosine_similarity
def recommend_movies(query, top_n=10):
    query_vec = vector.transform([query.lower()])
    query_vec_reduced = svd.transform(query_vec)
    print(query_vec)
    similarity = cosine_similarity(query_vec_reduced, vector_reduced).flatten()

    top_indices = similarity.argsort()[::-1][:top_n]
    recommended = movies.iloc[top_indices][['id','title','vote_average','vote_count','genres',"production_countries"]].copy()
    recommended['similarity'] = similarity[top_indices]
    return recommended


query_result = recommend_movies("Dilwale Dulhania Le Jayenge India romantic")
query_result.head(5)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 20000)>
  Coords	Values
  (0, 8928)	0.5070595636822558
  (0, 10187)	0.5972393991207468
  (0, 15238)	0.6214464570791676


,id,title,vote_average,vote_count,genres,production_countries,similarity
402486,1274006,jindagi jivi le,0.0,0,NaN,NaN,0.930368
1256654,808268,le pocha,0.0,0,documentary,india,0.863483
203544,1160225,nagumome chaalu le,6.5,2,"romance, drama",india,0.842076
173982,245134,le halua le,7.2,3,"comedy, drama, romance",india,0.835775
217452,558075,le chakka,5.5,2,comedy,india,0.835029
